In [ ]:
# Utilisation de gradio pour l'app avec SQLite + Docker ? https://www.youtube.com/watch?v=-GgG1Wb2F64&list=PLi0b6yHwHZcGaIy4ZGkYLZqssvzwqkBia&index=19&ab_channel=MASTER2SISEDATASCIENCE
# https://www.sqlite.org/docs.html
# Avantage -> Juste un pacakge, pas besoin d'installer un serveur, + facile pour déploiement docker
# Acces avec des fichiers et des tables à l'intérieurs (pysqlite doit être installé avec C++, CMake etc -> Trop compliqué avec docker, on va utiliser pysqlite3 wheels -> version déjà compilée)
# Compatbile avec streamlit mais démo avec gradio

In [ ]:
# TODO faire le sqlite + ajouter des champs dans le scrapping(clef étrangère)

: 

In [1]:
from scrapping_infos_resto import infos_resto
from scrapping_infos_avis import infos_avis # (url : str, max_pages : int = None) -> pd.DataFrame :

In [2]:
print("Test Function infos resto")
restau = infos_resto("https://www.tripadvisor.fr/Restaurant_Review-g187265-d19896976-Reviews-Empanadas_Club-Lyon_Rhone_Auvergne_Rhone_Alpes.html")


Test Function infos resto


In [17]:
help(infos_avis)

Help on function infos_avis in module scrapping_infos_avis:

infos_avis(url: str, max_pages: int = None) -> pandas.core.frame.DataFrame
    Scrape les informations des avis sur plusieurs pages à partir d'une URL de base.
    Arguments : 
        - url : une URL de la page web contenant les avis.
        - max_pages : Nombre maximum de pages à scraper.
    
    Retourne : 
        - Un data frame pandas contenant la date, la note et le commentaire de chaque avis



In [12]:
restau.head()

,Nom,Type_Cuisine,Fourchette_prix,Adresse,Note_moyenne
0,Empanadas Club,"Latino, Saine, Argentine, Sud-américaine","15,00€-25,00€","4 Rue Pizay Près De La Place Des Terreaux, 690...","5,0"


In [14]:
print("Test Function avis")
avis = infos_avis("https://www.tripadvisor.fr/Restaurant_Review-g187265-d19896976-Reviews-Empanadas_Club-Lyon_Rhone_Auvergne_Rhone_Alpes.html", 5)
print(avis.head())

Test Function avis
        Date Notes                                       Commentaires
0  nov. 2024   5,0  Un repas excellent servis rapidement dans une ...
1  déc. 2024   5,0  très bien pendant les fêtes des lumières et en...
2  nov. 2024   5,0  Très bel accueil et très belle variété d'empan...
3  nov. 2024   5,0  Très belle soirée passée entre amis dans ce li...
4  nov. 2024   5,0  Que ce soit sur le pouce avant une séance de c...


In [15]:
avis.head()


,Date,Notes,Commentaires
0,nov. 2024,"5,0",Un repas excellent servis rapidement dans une ...
1,déc. 2024,"5,0",très bien pendant les fêtes des lumières et en...
2,nov. 2024,"5,0",Très bel accueil et très belle variété d'empan...
3,nov. 2024,"5,0",Très belle soirée passée entre amis dans ce li...
4,nov. 2024,"5,0",Que ce soit sur le pouce avant une séance de c...


In [16]:
avis.to_csv("avis.csv", index = False)
restau.to_csv("restau.csv", index = False)

#### Création DB

In [4]:
import sqlite3

In [14]:
# Connexion à une base de données
conn = sqlite3.connect("restau.db") # Ouvrir la connexion


In [ ]:
# Créer un curseur pour exécuter des requêtes SQL
cursor = conn.cursor()

# Exemple de création de table
cursor.execute("""
CREATE TABLE restaurants (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nom TEXT NOT NULL,
    type_cuisine TEXT,
    fourchette_prix TEXT,
    adresse TEXT,
    note_moyenne REAL
)
""")

# Création de la table avis
cursor.execute("""
CREATE TABLE avis (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    id_restaurant INTEGER,
    date DATE,
    note INTEGER,
    commentaire TEXT,
    FOREIGN KEY (id_restaurant) REFERENCES restaurants(id)
)
""")

# Valider requêtes
conn.commit()

In [16]:
# Insertion de données en dur pour le moemnt
cursor.execute("""
INSERT INTO restaurants (nom, type_cuisine, fourchette_prix, adresse, note_moyenne)
VALUES ('Empanadas Club', 'Latino, Saine, Argentine, Sud-américaine', '15,00€-25,00', '4 Rue Pizay Près De La Place Des Terreaux, 69001 Lyon France', 5.0)
""")

# Valider requêtes
conn.commit()

In [17]:
cursor.execute("SELECT * FROM restaurants")
rows = cursor.fetchall()

for row in rows:
    print(row)



(1, 'Empanadas Club', 'Latino, Saine, Argentine, Sud-américaine', '15,00€-25,00', '4 Rue Pizay Près De La Place Des Terreaux, 69001 Lyon France', 5.0)


In [10]:
conn.close() # Fermer la connexion
